# Making measurements compatible with napari

This notebook explains how to handle the passage of napari-specific object classes between backend functions and napari layers. The key element of this passage is the `naparify_measurement` decorator (`napari_stress.measurements.utils.naparify_measurement`). It does the following things:

- It replaces the type annotation of a decorated function for type `manifold` with `napari.layers.Layer`. Thus, napari can create a widget from it.
- If a layer is passed to the function, the `manifold` object is retrieved from `layer.metadata` and forwarded to the function
- As measurement functions can return `data`, `features` or `metadata`, the resulting `features` and `metadata` are appended to the input `Layer`. If the function has been called with `manifold` as input rather than a `napari.layers.Layer`, the resulting `features` and `metadata` are returned.

In [4]:
from functools import wraps
import inspect
import napari
import numpy as np
from napari_stress._stress.manifold_SPB import manifold
import napari_stress

Let's demonstrate this with a simple example function:

In [2]:
def my_function(manifold: napari_stress._stress.manifold_SPB.manifold, sigma: float = 1.0) -> (dict, dict):
    some_data = np.random.random((10,3))
    metadata = {'attribute': 1}
    metadata['manifold'] = manifold
    features = {'attribute2': np.random.random(10)}
    return features, metadata

You'll see that this function expects a `manifold` object and a float number:

In [6]:
my_function

Decorating this function with `naparify_measurement` changes this:

In [9]:
napari_stress.measurements.utils.naparify_measurement(my_function)

<function __main__.my_function(manifold: 'napari.layers.Points', sigma: float = 1.0)>

## Demo with sample data

In [30]:
pointcloud = napari_stress.get_droplet_point_cloud()[0]
viewer.add_points(pointcloud[0][:, 1:], **pointcloud[1])

expansion = napari_stress.fit_spherical_harmonics(viewer.layers[-1].data)
viewer.add_points(expansion[0], **expansion[1])

lebedev_points = napari_stress._spherical_harmonics.spherical_harmonics_napari.perform_lebedev_quadrature(viewer.layers[-1], viewer=viewer)
results_layer = viewer.layers[-1]
assert 'manifold' in results_layer.metadata

`lebedev_points` is a Points layer with a `manifold` object stored within its `layer.metadata`:

In [31]:
results_layer.metadata['manifold']

We can now nicely demonstrate the behaviour of `naparify_measurement` - you'll see that the measured parameters `Mean_curvature_at_lebedev_points`, `H0_arithmetic_average` and `H0_surface_integral` have been appended to the layer:

In [35]:
napari_stress.measurements.calculate_mean_curvature_on_manifold(results_layer)
print('Metadata items:', results_layer.metadata.keys())
print('Features items:', results_layer.features.keys())

Old: <class 'napari_stress._stress.manifold_SPB.manifold'> New: napari.layers.Points
Metadata items: dict_keys(['spherical_harmonics_coefficients', 'spherical_harmonics_implementation', 'manifold', 'H0_arithmetic_average', 'H0_surface_integral'])
Features items: Index(['error', 'Mean_curvature_at_lebedev_points'], dtype='object')


If we were to pass the `manifold` object to the measurement function `calculate_mean_curvature_on_manifold()`, we'll directly receive the measured `features` and `metadata`:

In [39]:
_, features, metadata = napari_stress.measurements.calculate_mean_curvature_on_manifold(results_layer.metadata['manifold'])
print('Metadata items:', metadata.keys())
print('Features items:', features.keys())

Old: <class 'napari_stress._stress.manifold_SPB.manifold'> New: napari.layers.Points
Metadata items: dict_keys(['H0_arithmetic_average', 'H0_surface_integral'])
Features items: dict_keys(['Mean_curvature_at_lebedev_points'])
